In [1]:
from config import config, key
from train import train
import wandb

c:\Users\KaranjotVendal\mambaforge\envs\thesis\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\KaranjotVendal\mambaforge\envs\thesis\Lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [2]:
if config.WANDB:
    import wandb
    wandb.login(key=key.wandb_key)

In [3]:
if config.WANDB:
    r = ""
    wandb.init(
    project=f"{config.MODEL}_", 
    name=f"experiment_{config.MOD}_{r}",
    notes=f'using the finetune CNN on Brain MRIs {config.MODEL}_{config.MOD}.(Convx:LSTM:FC:classifier)',
    config={
    "learning_rate": config.LEARNING_RATE,
    "architecture": "CNN-GRU-MASK-FC-Classifier",
    "dataset": "MICAA MRI",
    "epochs": config.NUM_EPOCHS,
    "BATCH_SIZE": config.BATCH_SIZE,
    "Image Size": config.IMG_SIZE,
    "Modality": config.MOD
    })

In [4]:
#mod = ['FLAIR', 'T1w', 'T1wCE', 'T2w']
#for m in mod:
train()

--------------FOLD:1-----------------------
 Train Epoch: 1/15 | Loss: 0.69236 | Accuracy: 0.5419% | F1 Score: 0.4067 | AUROC: 0.5552 | Time: 115.87320923805237
Checkpoint saved at ./checkpoints/ConvxLSTM_model_FLAIR_1.pth | Test acc: 0.54% | Test F1: 0.516% | Best AUROC: 0.588
Validation Epoch: 0m 29s | Accuracy: 0.54% | F1 Score: 0.5164 | AUROC: 0.5878
 Train Epoch: 2/15 | Loss: 0.64467 | Accuracy: 0.6129% | F1 Score: 0.6078 | AUROC: 0.6820 | Time: 115.1017382144928
Validation Epoch: 0m 29s | Accuracy: 0.52% | F1 Score: 0.5084 | AUROC: 0.5823
 Train Epoch: 3/15 | Loss: 0.60423 | Accuracy: 0.6817% | F1 Score: 0.6813 | AUROC: 0.7539 | Time: 115.91717982292175
Checkpoint saved at ./checkpoints/ConvxLSTM_model_FLAIR_1.pth | Test acc: 0.60% | Test F1: 0.574% | Best AUROC: 0.599
Validation Epoch: 0m 30s | Accuracy: 0.60% | F1 Score: 0.5739 | AUROC: 0.5987
 Train Epoch: 4/15 | Loss: 0.51332 | Accuracy: 0.8194% | F1 Score: 0.8193 | AUROC: 0.9100 | Time: 114.45308995246887
Validation Epoch: 0